In [9]:
import os

In [10]:
import astropy.stats as apy

In [11]:
import bk.load
import bk.compute
import bk.plot
import bk.signal

In [12]:
import pandas as pd
import numpy as np
import scipy as sp

In [13]:
import matplotlib.pyplot as plt

In [14]:
import scipy.signal
import scipy.stats
%load_ext memory_profiler

In [15]:
def concentration(angles):
    '''
    Compute the kappa parameter Uses the approximation described in "Statistical Analysis of Circular Data" (Fisher, p. 88).
    Translated from MATLAB fmatoolbox.sourceforge.net/Contents/FMAToolbox/General/Concentration.html
    angles : radian

    Copyright (C) 2004-2011 by Michaël Zugaro
    Copyright (C) 2021 by Billel KHOUADER

    '''
    n = len(angles)
    angles = np.exp(1j * angles)  # Complex form of the angles
    r = abs(np.mean(angles))

    if r < 0.53:
        k = 2 * r + r**3 + 5*r**(5/6)
    elif r < 0.85:
        k = -0.4 + 1.39 * r + 0.43 / (1-r)
    else:
        k = 1/(r**3 - 4 * r**2 + 3*r)

    # Correction for very small samples

    if n <= 15:
        if k < 2:
            k = np.max([(k-2)/(n*k), 0])
        else:
            k = (n-1)**3 * (n**3+n)

    return k


def mean_resultant_length(angles):
    angles = np.exp(1j*angles)
    return np.abs(np.mean(angles))

In [16]:
def thetamod(local_path, **args):
    bk.load.current_session_linux(local_path=local_path)
    states = bk.load.states()
    lfp = bk.load.lfp_in_intervals(bk.load.ripple_channel(), states['Rem'])
    lfp_filt = bk.signal.passband(lfp, 4, 12)
    power, phase = bk.signal.hilbert(lfp_filt)

    neurons, metadata = bk.load.spikes()
    print(os.getcwd())


    neurons_phase = []
    neurons_rem_fr = []
    for i, n in enumerate(neurons):
        neurons_phase.append(phase.realign(n.restrict(states['Rem'])))
        neurons_rem_fr.append(
            len(n.restrict(states['Rem']))/states['Rem'].tot_length(time_units='s'))

    pvalues = []
    kappas = []
    modulated = []
    MRLs = []
    for i, p in enumerate(neurons_phase):
        pvalue = apy.rayleightest(p.values)
        pvalues.append(pvalue)

        kappa = concentration(p.values)
        kappas.append(kappa)

        MRL = mean_resultant_length(p.values)
        MRLs.append(MRL)

        if (pvalue < 0.01) and MRL > 0.04:
            modulated.append(True)
        else:
            modulated.append(False)

    phase_lock = pd.DataFrame({'FR_Rem': neurons_rem_fr,
                               'MRL': MRLs,
                               'Kappa': kappas,
                               'pValue': pvalues,
                               'modulated': modulated})

    # pvalues = pd.DataFrame(pvalues, columns=['pValue'])
    # FRRem = pd.DataFrame(neurons_rem_fr, columns=['FRRem'])

    phase_lock = pd.concat([metadata, phase_lock], axis=1)
    
    try:
        os.makedirs(f'Analysis/Thetamod')
    except:
        print('Already Exist')
    phase_lock.to_csv(
        f'Analysis/Thetamod/{bk.load.session}-theta_mod.csv')
    
    # for i in range(len(neurons)):
    #     fig, ax = plt.subplot_mosaic('''
    #                                 A.
    #                                 BC
    #                                 ''',
    #                                  figsize=(12, 8))
    #     fig.set_visible = False
    #     polar_ax = fig.add_subplot(2, 2, 2, polar=True)
    #     ax['A'].plot(lfp.as_units('s'))
    #     ax['A'].plot(lfp_filt.as_units('s'))
    #     plt.sca(ax['A'])
    #     bk.plot.intervals(states['Rem'], time_units='s')
    #     ax['B'].plot(phase.as_units('s'))
    #     ax['B'].plot(neurons_phase[i].as_units('s'), 'ro')

    #     h, a = np.histogram(neurons_phase[i].values, 50)
    #     a = np.convolve(a, [.5, .5], 'same')[1::]

    #     ax['C'].bar(a, h, a[1]-a[0], color='lightblue')
    #     ax['C'].bar(a+2*np.pi, h, a[1]-a[0], color='lightblue')

    # #     ax['C'].bar(neurons_phase[i].values+(2*np.pi),50,'blue')

    #     ax['A'].get_shared_x_axes().join(ax['A'], ax['B'])
    #     polar_ax.bar(a, h, a[1]-a[0])

    #     ax['A'].set_title('LFP')
    #     ax['A'].set_ylim([-500, 500])
    #     ax['A'].set_xlabel('Time (s)')
    #     ax['A'].set_ylabel('Amplitude')

    #     ax['B'].set_title('Phase And Spikes')
    #     ax['B'].set_xlabel('Time (s)')
    #     ax['B'].set_ylabel('Phase (rad)')
    #     polar_ax.set_title('PolarPlot')

    #     ax['C'].set_title('Histogram')
    #     ax['C'].set_xlabel('Phase (rad)')
    #     ax['C'].set_ylabel('Spike Count')

    #     if phase_lock.modulated[i]:
    #         fig.set_facecolor('lightgreen')

    #     plt.suptitle(
    #         f'Neuron #{i},Region = {phase_lock.Region[i]}, Type = {phase_lock.Type[i]}  \nFR_Rem = {phase_lock.FR_Rem[i]} Hz \npValue = {phase_lock.pValue[i]} \n MRL = {phase_lock.MRL[i]}, Kappa = {phase_lock.Kappa[i]} \n Modulated : {phase_lock.modulated[i]}')
    #     plt.tight_layout()

    #     plt.savefig(
    #         f'/home/billel/figs/ThetaMod/{bk.load.session}/Neuron #{i}-{metadata.Shank[i]}-{metadata.Id[i]}.png')
    #     fig.clear()
    #     plt.close(fig)

    del lfp, lfp_filt, neurons, neurons_phase
    return phase_lock


In [17]:
%matplotlib Agg

In [18]:
b = bk.load.batch(thetamod,linux=True)

100%|██████████| 61/61 [07:28<00:00,  7.35s/it]

Batch finished in 448.37303829193115
Some session were not processed correctly
['Rat08-20130718', 'Rat08-20130719', 'Rat08-20130720', 'Rat08-20130722', 'Rat09-20140324', 'Rat09-20140325', 'Rat09-20140326', 'Rat09-20140327', 'Rat09-20140328', 'Rat09-20140329', 'Rat09-20140331', 'Rat09-20140401', 'Rat09-20140402', 'Rat09-20140403', 'Rat09-20140404', 'Rat09-20140405', 'Rat09-20140407', 'Rat09-20140408', 'Rat09-20140409', 'Rat11-20150327']
32.78688524590164 %


In [15]:
b

{}

In [13]:
phase_locks = []
for session,phase_lock in b.items():
    phase_locks.append(phase_lock)

In [3]:
os.getcwd()
os.chdir('/mnt/icybox/Rat08/Rat08-20130713/')

In [6]:
os.makedirs(f'Analysis/Thetamod')


In [14]:
phase_locks = pd.concat(phase_locks)

ValueError: No objects to concatenate

In [26]:
phase_locks.to_csv("/home/billel/figs/ThetaMod/ThetaMod.csv")

In [4]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib

for i in range(10):
    plt.subplot_mosaic('A',visible = False)
    np.sqrt(i)
    plt.plot(i)
print('yes')

Using matplotlib backend: Qt5Agg


TypeError: __init__() got an unexpected keyword argument 'visible'